In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import log, sqrt
from sklearn import linear_model  # using scikit-learn
%matplotlib inline

# Part 1

In [150]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('hw1/kc_house_data.csv', dtype=dtype_dict)

In [151]:
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']
sales['floors_square'] = sales['floors']*sales['floors']

In [152]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

## Q1

In [153]:
print [all_features[i] for i in range(len(all_features)) if model_all.coef_[i] != 0]

['sqft_living', 'view', 'grade']


## Q2

In [154]:
testing = pd.read_csv('hw1/wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('hw1/wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('hw1/wk3_kc_house_valid_data.csv', dtype=dtype_dict)

In [155]:
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

In [156]:
l1_penalty = np.logspace(1, 7, num=13)
rss = []

In [157]:
for l1 in l1_penalty:
    model = linear_model.Lasso(alpha=l1, normalize=True)
    model.fit(training[all_features], training['price'])
    rss.append(np.sum((validation['price'] - model.predict(validation[all_features]))**2))
    print rss[-1]
print 'l1 penaly achieving minimum validation error = ' + str(l1_penalty[rss.index(min(rss))])

3.982133273e+14
3.99041900253e+14
4.29791604073e+14
4.63739831045e+14
6.45898733634e+14
1.22250685943e+15
1.22250685943e+15
1.22250685943e+15
1.22250685943e+15
1.22250685943e+15
1.22250685943e+15
1.22250685943e+15
1.22250685943e+15
l1 penaly achieving minimum validation error = 10.0


## Q3

In [158]:
best_l1 = l1_penalty[rss.index(min(rss))]
best_model = linear_model.Lasso(alpha=best_l1, normalize=True)
best_model.fit(training[all_features], training['price'])
print str(np.count_nonzero(best_model.coef_) + np.count_nonzero(best_model.intercept_))

15


## Q4

In [159]:
num_nonzero = []
l1_penalty = np.logspace(1, 4, num=20)
max_nonzeros = 7
for l1 in l1_penalty:
    model = linear_model.Lasso(alpha=l1, normalize=True)
    model.fit(training[all_features], training['price'])
    num_nonzero.append(np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_))
print num_nonzero
l1_penalty_max = [l1_penalty[i] for i in range(len(l1_penalty)) if num_nonzero[i] < max_nonzeros][0]
l1_penalty_min = [l1_penalty[i] for i in range(len(l1_penalty)) if num_nonzero[i] > max_nonzeros][-1]
print 'l1 max = ' + str(l1_penalty_max) + ' l1 min = ' + str(l1_penalty_min)

[15, 15, 15, 15, 13, 12, 11, 10, 7, 6, 6, 6, 5, 3, 3, 2, 1, 1, 1, 1]
l1 max = 263.665089873 l1 min = 127.42749857


In [160]:
print l1_penalty

[    10.             14.38449888     20.69138081     29.76351442
     42.81332399     61.58482111     88.58667904    127.42749857
    183.29807108    263.66508987    379.26901907    545.55947812
    784.75997035   1128.83789168   1623.77673919   2335.72146909
   3359.81828628   4832.93023857   6951.92796178  10000.        ]


## Q5

In [161]:
l1_penalty = np.linspace(l1_penalty_min,l1_penalty_max,20)
num_nonzero = []
rss = []
for l1 in l1_penalty:
    model = linear_model.Lasso(alpha=l1, normalize=True)
    model.fit(training[all_features], training['price'])
    rss.append(np.sum((validation['price'] - model.predict(validation[all_features]))**2))
    num_nonzero.append(np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_))

min_rss = min([rss[i] for i in range(len(l1_penalty)) if num_nonzero[i] == max_nonzeros])
best_l1_penalty = l1_penalty[rss.index(min_rss)]
print 'min rss = ' + str(min_rss) + ' best l1 penalty = ' + str(best_l1_penalty) 

min rss = 4.40037365263e+14 best l1 penalty = 156.109096739


## Q6

In [162]:
model = linear_model.Lasso(alpha=best_l1_penalty, normalize=True)
model.fit(training[all_features], training['price'])
print [all_features[i] for i in range(len(all_features))  if model.coef_[i] != 0 ]
print model.intercept_

['bathrooms', 'sqft_living', 'waterfront', 'view', 'grade', 'yr_built']
4422190.27912


In [163]:
#

In [164]:
#

In [165]:
#

# Part 2

In [166]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

data = pd.read_csv('hw2/kc_house_data.csv', dtype=dtype_dict)
train = pd.read_csv('hw2/kc_house_train_data.csv', dtype=dtype_dict)
test = pd.read_csv('hw2/kc_house_test_data.csv',dtype=dtype_dict)

In [167]:
def get_numpy_data(data, features, output):
    data['constant'] = 1
    features = ['constant'] + features
    output_np = data[output]
    output_np = output_np.as_matrix()
    data_np = data.as_matrix(features)
    return (data_np, output_np)

In [168]:
def predict_output(data_np, weights):
    predictions = np.dot(data_np, weights)
    return predictions

In [169]:
def normalize_features(features):
    norms = np.linalg.norm(features, axis=0)
    normalized_features = features/norms
    return (normalized_features, norms)

In [170]:
def compute_rho_i(data_np, output_np, weights, i):
    predictions = np.dot(data_np, weights)
    ro_i = np.dot(data_np[:, i], (output_np - predictions + weights[i]*data_np[:, i]))
    return ro_i

## Q1 && Q2

In [171]:
initial_weight = [1, 4, 1]
features = ['sqft_living', 'bedrooms']
output = 'price'

data_np, output_np = get_numpy_data(data, features, output)
normalized_data_np, norms = normalize_features(data_np)

In [172]:
print 'higher bound of lambda = ' + str(2*compute_rho_i(normalized_data_np, output_np, initial_weight, 1))
print 'lower bound of lambda ' + str(2*compute_rho_i(normalized_data_np, output_np, initial_weight, 2))

higher bound of lambda = 175878941.647
lower bound of lambda 161933397.332


## Q3

In [173]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    # prediction = ...
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = compute_rho_i(feature_matrix, output, weights, i)
    
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

In [174]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                   [2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


In [175]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, weights, l1_penalty, tolerance):
    converged = False
    while not converged:
        change_in_weight = []
        for i in range(len(weights)):
            old_weight = weights[i]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            change_in_weight.append(abs(weights[i] - old_weight))
        if max(change_in_weight) < tolerance:
            converged = True
    return weights

In [176]:
features = ['sqft_living', 'bedrooms']
output = 'price'

data_np, output_np = get_numpy_data(data, features, output)
normalized_data_np, norms = normalize_features(data_np)

initial_weights = [0, 0, 0]
l1_penalty = 1e7
tolerance = 1.0

lasso_model = lasso_cyclical_coordinate_descent(normalized_data_np, output_np, initial_weights, l1_penalty, tolerance)

print 'rss = ' + str(np.sum((output_np - predict_output(normalized_data_np, lasso_model))**2))
print lasso_model

rss = 1.63049247672e+15
[21624997.959519144, 63157247.207889512, 0.0]


## Q4

In [180]:
train = pd.read_csv('hw2/kc_house_train_data.csv', dtype=dtype_dict)
test = pd.read_csv('hw2/kc_house_test_data.csv', dtype=dtype_dict)

features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']
output = 'price'

train_np, train_output_np = get_numpy_data(train, features, output)
normalized_train_np, norms = normalize_features(train_np)

initial_weights = np.zeros(len(features)+1)
l1_penalty = 1e7
tolerance = 1.0

weights1e7 = lasso_cyclical_coordinate_descent(normalized_train_np, train_output_np, initial_weights, l1_penalty, tolerance)
print [features[i] for i in range(len(features)) if weights1e7[i+1] != 0]

initial_weights = np.zeros(len(features)+1)
l1_penalty = 1e8
weights1e8 = lasso_cyclical_coordinate_descent(normalized_train_np, train_output_np, initial_weights, l1_penalty, tolerance)
print [features[i] for i in range(len(features)) if weights1e8[i+1] != 0]

initial_weights = np.zeros(len(features)+1)
l1_penalty = 1e4
tolerance = 5e5
weights1e4 = lasso_cyclical_coordinate_descent(normalized_train_np, train_output_np, initial_weights, l1_penalty, tolerance)
print [features[i] for i in range(len(features)) if weights1e4[i+1] != 0]

['sqft_living', 'waterfront', 'view']
[]
['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']


## Q5

In [181]:
normalized_weights1e7 = weights1e7/norms
normalized_weights1e8 = weights1e8/norms
normalized_weights1e4 = weights1e4/norms

print normalized_weights1e7[3]

161.317457646


In [182]:
test_np, test_out_np = get_numpy_data(test, features, output)

In [184]:
print 'rss of l1 = 1e7 is: ' + str(np.sum((test_out_np - predict_output(test_np, normalized_weights1e7))**2))
print 'rss of l1 = 1e8 is: ' + str(np.sum((test_out_np - predict_output(test_np, normalized_weights1e8))**2))
print 'rss of l1 = 1e4 is: ' + str(np.sum((test_out_np - predict_output(test_np, normalized_weights1e4))**2))

rss of l1 = 1e7 is: 2.7596207592e+14
rss of l1 = 1e8 is: 5.37166151497e+14
rss of l1 = 1e4 is: 2.28459958971e+14
